In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models,layers
from imgaug import augmenters as iaa
import imgaug as ia
from sklearn.metrics import classification_report
import itertools

In [ ]:
#carregando as imagens
df_train = pd.read_csv('../input/pastas-de-dados/df_train.csv')
df_test = pd.read_csv('../input/pastas-de-dados/df_test.csv')
df_val = pd.read_csv('../input/pastas-de-dados/df_val.csv')

base_dir = '../input/isic-2019-challenge/ISIC_2019_Training_Input/ISIC_2019_Training_Input'

In [ ]:
#carregando as imagens para a classe binaria
df_trainBin = pd.read_csv('../input/pastas-de-dados/df_train.csv')
df_testBin = pd.read_csv('../input/pastas-de-dados/df_test.csv')
df_valBin = pd.read_csv('../input/pastas-de-dados/df_val.csv')

df_trainBin = df_trainBin.assign(label=0)
df_trainBin.loc[df_trainBin['MEL']==1.0, 'label'] = 1
df_trainBin.loc[df_trainBin['BCC']==1.0, 'label'] = 1
df_trainBin.loc[df_trainBin['VASC']==1.0, 'label'] = 1
df_trainBin.loc[df_trainBin['SCC']==1.0, 'label'] = 1

df_valBin = df_valBin.assign(label=0)
df_valBin.loc[df_valBin['MEL']==1.0, 'label'] = 1
df_valBin.loc[df_valBin['BCC']==1.0, 'label'] = 1
df_valBin.loc[df_valBin['VASC']==1.0, 'label'] = 1
df_valBin.loc[df_valBin['SCC']==1.0, 'label'] = 1

df_testBin = df_testBin.assign(label=0)
df_test.loc[df_testBin['MEL']==1.0, 'label'] = 1
df_test.loc[df_testBin['BCC']==1.0, 'label'] = 1
df_test.loc[df_testBin['VASC']==1.0, 'label'] = 1
df_test.loc[df_testBin['SCC']==1.0, 'label'] = 1

In [ ]:
#Rand_argument
tf.random.set_seed(42)
ia.seed(42)

rand_aug = iaa.RandAugment(n=1, m=2)

def augment(image):
    image = tf.cast(image, tf.uint8)
    return rand_aug(image=image.numpy())

#recorte aleatório
def random_crop(image,aspect_ratio_range=[0.75, 1.33],area_range=[0.08, 1],min_object_covered=0.5,max_attempts=100,seed=0):
    bbox = tf.constant([0.0, 0.0, 1.0, 1.0], dtype=tf.float32, shape=[1, 1, 4])
    bbox_begin, bbox_size, _ = tf.image.sample_distorted_bounding_box(
    tf.shape(image),
    bounding_boxes=bbox,
    min_object_covered=min_object_covered,
    area_range=area_range,
    aspect_ratio_range=aspect_ratio_range,
    use_image_if_no_bounding_boxes=True,
    max_attempts=max_attempts,
    seed=seed)
    offset_height, offset_width, _ = tf.unstack(bbox_begin)
    target_height, target_width, _ = tf.unstack(bbox_size)

    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.crop_to_bounding_box(image,offset_height,offset_width,target_height,target_width)

    return image

In [ ]:
def loadImage(image,mel,nv,bcc,ak,bkl,df,vasc,scc):
  image = tf.io.read_file(base_dir + '/' + image + '.jpg')
  image = tf.io.decode_jpeg(image,channels=3)
  label = tf.stack([mel,nv,bcc,ak,bkl,df,vasc,scc])
  return image,label

def loadImageBin(image,label):
  image = tf.io.read_file(base_dir + '/' + image + '.jpg')
  image = tf.io.decode_jpeg(image,channels=3)
 
  return image,label

def dataAument(image,label):
  image = random_crop(image)
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.resize(image,size = (260,260)) #EfficientNetB0
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  image_shape = tf.shape(image)
  image = tf.image.convert_image_dtype(image, dtype=tf.uint8)
  image2 = tf.py_function(augment,[image],[tf.float32])[0]
  image = tf.ensure_shape(image2,shape=image.get_shape())

  return image,label
 
def resize(image,label):
  image = tf.image.resize(image,size = (260,260)) #EfficientNetB0
  
  return image,label

In [ ]:
dataSet_train = tf.data.Dataset.from_tensor_slices((df_train['image'],
                                                    df_train['MEL'],
                                                    df_train['NV'],
                                                    df_train['BCC'],
                                                    df_train['AK'],
                                                    df_train['BKL'],
                                                    df_train['DF'],
                                                    df_train['VASC'],
                                                    df_train['SCC']))
dataSet_train = dataSet_train.shuffle(len(df_train))
dataSet_train = dataSet_train.map(loadImage)
dataSet_train = dataSet_train.map(dataAument)

dataSet_train = dataSet_train.batch(64)

In [ ]:
dataSet_val = tf.data.Dataset.from_tensor_slices((df_val['image'],
                                                    df_val['MEL'],
                                                    df_val['NV'],
                                                    df_val['BCC'],
                                                    df_val['AK'],
                                                    df_val['BKL'],
                                                    df_val['DF'],
                                                    df_val['VASC'],
                                                   df_val['SCC']))
dataSet_val = dataSet_val.map(loadImage)
dataSet_val = dataSet_val.map(resize)
dataSet_val = dataSet_val.batch(64)

In [ ]:
dataSet_test = tf.data.Dataset.from_tensor_slices((df_test['image'],
                                                    df_test['MEL'],
                                                    df_test['NV'],
                                                    df_test['BCC'],
                                                    df_test['AK'],
                                                    df_test['BKL'],
                                                    df_test['DF'],
                                                    df_test['VASC'],
                                                   df_test['SCC']))
dataSet_test = dataSet_test.map(loadImage)
dataSet_test = dataSet_test.map(resize)
dataSet_test = dataSet_test.batch(64)

In [ ]:
#Criando os dataSets binarios
dataSet_trainBin = tf.data.Dataset.from_tensor_slices((df_trainBin['image'],df_trainBin['label']))
dataSet_trainBin = dataSet_trainBin.shuffle(len(df_trainBin))
dataSet_trainBin = dataSet_trainBin.map(loadImageBin)
dataSet_trainBin = dataSet_trainBin.map(dataAument)
dataSet_trainBin = dataSet_trainBin.batch(64)

dataSet_valBin = tf.data.Dataset.from_tensor_slices((df_valBin['image'],df_valBin['label']))
dataSet_valBin = dataSet_valBin.map(loadImageBin)
dataSet_valBin = dataSet_valBin.map(resize)
dataSet_valBin = dataSet_valBin.batch(64)

dataSet_testBin = tf.data.Dataset.from_tensor_slices((df_valBin['image'],df_valBin['label']))
dataSet_testBin = dataSet_testBin.map(loadImageBin)
dataSet_testBin = dataSet_testBin.map(resize)
dataSet_testBin = dataSet_testBin.batch(64)

In [ ]:
image_input = tf.keras.Input(shape=(260,260,3))
base_modelBin = EfficientNetB2(include_top=False,weights='imagenet')

base_modelBin._name = 'efficientnetb0_bin'
x = base_modelBin(image_input, training=True)
x = layers.GlobalAveragePooling2D()(x)
outputs_bin = tf.keras.layers.Dense(1,activation='sigmoid',name='binary')(x)
model_bin = tf.keras.models.Model(inputs=image_input, outputs=outputs_bin)

model_bin.summary()

In [ ]:
#Treino do modelo binario
model_bin.compile(loss="binary_crossentropy",optimizer='adam',metrics=["acc"])

H1 = model_bin.fit(dataSet_trainBin,epochs=30,validation_data = dataSet_valBin)

In [ ]:
model_bin.trainable = False

In [ ]:
base_model = EfficientNetB2(include_top=False,weights='imagenet')

base_model._name = 'efficientnetb0_full'

x = base_model(image_input, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Concatenate()([x, outputs_bin])
outputs = tf.keras.layers.Dense(8,activation='softmax')(x)

model = tf.keras.models.Model(inputs=image_input, outputs=outputs)

model.summary()

In [ ]:
#Treino das duas
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["acc"])

H2 = model.fit(dataSet_train,epochs=30,validation_data = dataSet_val)

In [ ]:
results = model.evaluate(dataSet_test)
print("test loss, test acc:", results)

In [ ]:
#true labels
df_true = pd.DataFrame([x for x in np.where(df_test == 1, df_test.columns,'').flatten().tolist() if len(x) > 0], columns= (["dx"]))
true_labels = []
for i in df_true['dx']:
  true_labels.append(i)

n = 0 
for i in true_labels:
  if i == 'MEL':
    true_labels[n] = 0
  if i == 'NV':
    true_labels[n] = 1
  if i == 'BCC':
    true_labels[n] = 2
  if i == 'AK':
    true_labels[n] = 3
  if i == 'BKL':
    true_labels[n] = 4
  if i == 'DF':
    true_labels[n] = 5
  if i == 'VASC':
    true_labels[n] = 6
  if i == 'SCC':
    true_labels[n] = 7
  n += 1

In [ ]:
predictions = tf.math.argmax(model.predict(dataSet_test),axis=1)

In [ ]:
vet_labels = ['MEL','NV','BCC','AK','BKL','DF','VASC','SCC']

In [ ]:
print(classification_report(true_labels,predictions,target_names=vet_labels))